# Span Batch Analysis - Goerli Data Preparation

### Batch Decoder

Build `batch_decoder` binary from [pcw109550/batch-decoder-parallel-tx-fetch](https://github.com/testinprod-io/optimism/tree/pcw109550/batch-decoder-parallel-tx-fetch) branch.

In [ ]:
!git clone git@github.com:testinprod-io/optimism.git

In [ ]:
!git switch pcw109550/batch-decoder-parallel-tx-fetch
!cd optimism/op-node/cmd/batch_decoder
!go build && cd -

Now we have `batch_decoder` binary at `op-node/cmd/batch_decoder/batch_decoder`.

### Batch Decoder `fetch`

Read channel frames from L1 by reading batch transactions. L1 block range: `[9500000,9520000)`

In [ ]:
!time ./cmd/batch_decoder/batch_decoder fetch --l1=[L1_GOERLI_RPC] --start=9500000  --end=9520000  --inbox=0xff00000000000000000000000000000000000420 --out=./artifacts/goerli/transactions_cache  --sender=0x7431310e026B69BFC676C0013E12A1A11411EEc9 --concurrent-requests=25 

### Batch Decoder `reassemble`

Reassemble channel frames to channels. 

In [ ]:
!time ./cmd/batch_decoder/batch_decoder reassemble --inbox=0xff00000000000000000000000000000000000420 --in=./artifacts/goerli/transactions_cache --out=./artifacts/goerli/channel_cache

### Span Batch Tester

Build `span_batch_tester` binary with tx encoding selection enabled from [pcw109550/span-batch-tester-with-tx-encoding](https://github.com/testinprod-io/optimism/tree/pcw109550/span-batch-tester-with-tx-encoding) branch.

In [ ]:
!git switch pcw109550/span-batch-tester-with-tx-encoding
!cd optimism/op-node/cmd/span_batch_tester
!go build && cd -

Now we have `span_batch_tester` binary at `op-node/cmd/span_batch_tester/span_batch_tester`.

### Span Batch Tester `convert`

Convert channels with v0 batches to span batches. Try multiple tx encoding types.

#### `BatchV2TxsV1Type`

In [ ]:
!time ./cmd/span_batch_tester/span_batch_tester convert --in=./artifacts/goerli/channel_cache --out=./artifacts/goerli/span_batch_cache_v1 --l2=[L2_GOERLI_RPC] --genesis-timestamp=1673550516 --chain-id=420 --tx-type=0

#### `BatchV2TxsV2Type`

In [ ]:
!time ./cmd/span_batch_tester/span_batch_tester convert --in=./artifacts/goerli/channel_cache --out=./artifacts/goerli/span_batch_cache_v2 --l2=[L2_GOERLI_RPC] --genesis-timestamp=1673550516 --chain-id=420 --tx-type=1

#### `BatchV2TxsV3Type`

In [ ]:
!time ./cmd/span_batch_tester/span_batch_tester convert --in=./artifacts/goerli/channel_cache --out=./artifacts/goerli/span_batch_cache_v3 --l2=[L2_GOERLI_RPC] --genesis-timestamp=1673550516 --chain-id=420 --tx-type=2

### Span Batch Tester `analyze`

Analyze channels with v0 batches by comparing with corresponding span batches.

#### `BatchV2TxsV1Type`

In [ ]:
!time ./cmd/span_batch_tester/span_batch_tester analyze --in-channel=./artifacts/goerli/channel_cache --in-span-batch=./artifacts/goerli/span_batch_cache_v1 --out=./artifacts/goerli/result_v1

#### `BatchV2TxsV2Type`

In [ ]:
!time ./cmd/span_batch_tester/span_batch_tester analyze --in-channel=./artifacts/goerli/channel_cache --in-span-batch=./artifacts/goerli/span_batch_cache_v2 --out=./artifacts/goerli/result_v2

#### `BatchV2TxsV3Type`

In [ ]:
!time ./cmd/span_batch_tester/span_batch_tester analyze --in-channel=./artifacts/goerli/channel_cache --in-span-batch=./artifacts/goerli/span_batch_cache_v3 --out=./artifacts/goerli/result_v3